In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import MCMT
import numpy as np
from scipy.stats import unitary_group
from qiskit.quantum_info import Statevector
from qiskit.extensions.unitary import UnitaryGate
from qiskit.converters import circuit_to_gate

In [79]:
from qiskit.circuit import Parameter, Gate, ParameterVector

#example for adding parameterized gate
def U_H(theta):
    circuit = QuantumCircuit(2)

    # Global phase:
    circuit.x(0)
    circuit.u(np.pi, -theta, np.pi - theta, 0)

    circuit.rxx(2 * theta, 0, 1)
    circuit.ryy(2 * theta, 0, 1)
    circuit.rzz(2 * theta, 0, 1)
    return circuit.to_gate(label = 'U_1')

theta = Parameter('θ')
circ = QuantumCircuit(2)
circ.append(U_H(theta), [0, 1])
bound_circuit = circ.bind_parameters({ theta: np.pi / 16 })

circ.draw()

┌─────────┐
q_0: ┤0        ├
     │  U_1(θ) │
q_1: ┤1        ├
     └─────────┘

Generating 4x4 parameterized unitary

In [50]:
# uses expession from https://arxiv.org/abs/1103.3408 

from scipy.linalg import expm

def P(i):
    mat = np.zeros([4,4],'complex128')
    mat[i,i] = 1
    return mat

def Y(k,l):
    mat = np.zeros([4,4],'complex128')
    mat[k,l] = -1j
    mat[l,k] = 1j
    return mat

# return 4x4 unitary parameterized by (4^2)-1 = 15 angles in theta array
def SU_4(theta):
    mat = np.identity(4,'complex128')
    for i in range(3):
        for j in [0,1,2,3][i+1:]:
            mat = mat @ (expm(1j*P(j)*theta[j,i]) @ expm(1j*Y(i,j)*theta[i,j]))
    for i in range(4):
        mat = mat @ expm(1j*P(i)*theta[i,i])
    return mat

In [78]:
from numpy.random import uniform
# random theta, upper diagonal must be [0,pi/2] phase shift, lower diagonal [0,2pi] rotations.

def rand_theta():
    theta = np.zeros((4,4))
    for i in range(4):
        for j in [0,1,2,3][i:]:
            theta[j,i] = uniform(0,2*np.pi)
        for j in [0,1,2,3][:i]:
            theta[j,i] = uniform(0,np.pi/2)
    return theta

In [93]:
# check that it produces unitary, seems to be true
def is_unitary(m):
    return np.allclose(np.eye(m.shape[0]), m.H * m)
M = np.matrix(SU_4(rand_theta()))
is_unitary(M)

True

It complains that parameters are not subscriptable!

In [104]:
from qiskit.quantum_info.operators import Operator, Pauli

# function to create controlled gate, k is position of control bits, theta is unitary parameters
def controlled_U(k,theta):
    circuit = QuantumCircuit(4)
    mat = np.kron(P(k),SU_4(theta))
    for i in (i for i in [0,1,2,3] if i != k):
        mat = mat + np.kron(P(i),np.eye(4)) 
    gate = UnitaryGate(mat)
    return gate

# adding gate to circuit with parameters (8 qubits = two sequences)
theta1 = ParameterVector('lamda1',16)
circ = QuantumCircuit(8)
circ.append(controlled_U(1,theta1), [0, 1, 4, 5])
#bound_circuit = circ.bind_parameters({ theta: np.pi / 16 })

circ.draw()


TypeError: '>' not supported between instances of 'tuple' and 'int'

## Ignore all code below this point

In [83]:
gen = (x for x in [0,1,2,3] if x != 1)

for x in (x for x in [0,1,2,3] if x != 1):
    print(x)

0
2
3


In [ ]:
k_ = 4                        # number of hidden states 
k = int(np.ceil(np.log2(k_))) # first wire qubits
s_ = 4                        # number of observable states
s = int(np.ceil(np.log2(s_))) # second wire qubits
n = 2                         # number of variables

num_qubits = n*(k+s)
gap = k+s
nums = np.arange(num_qubits)

In [189]:
def cont_gate():
    mat = unitary_group.rvs(k_)
    g = UnitaryGate(mat).control(k)
    return g

def single_gate(m):
    mat = unitary_group.rvs(m)
    g = UnitaryGate(mat)
    return g

In [148]:
# integer to binary representation in array form
def bin_array(num):
    return np.array(list(np.binary_repr(num).zfill(2))).astype(np.int8)

In [7]:
nums = np.arange(num_qubits)

def add_controlled_gates(c):
    # getting indexes
    c,t = c*gap, (c+1)*gap
    control = (nums[c:])[:k]
    target = (nums[(t+k):])[:s]
    indexes = np.concatenate((control,target))

    # apply multi target control gates
    for i in range(k_):
        b = bin_array(i)
        for j in range(k):
            if b[j] == 1:
                qc.x(c+j)
        qc.append(cont_gate(), indexes)
        for j in b:
            if b[j] == 1:
                qc.x(c+j)

def add_prior(t):
    qc.append(single_gate(k),t) 

def add_basischange(t):
    qc.append(single_gate(n),t)

def build_control_gate():
    circ = QuantumCircuit(2*k)

    def x_gates(b):
        for j in b:
            if j == 1:
                circ.x(j)

    for i in range(k_):
        b = bin_array(i)
        x_gates(b)
        qc.append(cont_gate(), [0,1,2,3])
        x_gates(b)
    
    g = circuit_to_gate(circ)
    return g


NameError: name 'num_qubits' is not defined